In [1]:
# all things geodesic including transformations between standard 
# geospatial coordinate systems
using Geodesy

# for direct manipulations of Cartesian and Longitude-Latitude coordinates
using CoordinateTransformations, Rotations, StaticArrays

include("mathies.jl")




"mathies.jl inclusion concluded"

In [2]:
# generate the coordinates for global positioning centerpoints (beads)
#   icosohedral vertices are centerpoints of the 12 dodecahedral regions 


# icoso_verts points have an edge not a vertex at top and bottom;
# rotate -31.71 degrees ( -( pi/2 - atan(Φ) ) ) on the X axis
rotateX  = LinearMap( RotX( -( pi/2 - atan(Φ)) ) )
# that results in untidy longitudes; spin eastward 18 degrees
rotateZ  = LinearMap( RotZ(pi/10) )
# create a composite rotation
rotate = rotateZ ∘ rotateX 
points = SVector.(rotate.(GEO_ICOSO_VERTS))



# generate LLA's for each point


# radius here is the distance of the vertex from the icosohedron center
icoso_circumradius = sqrt(Φ^2 + 1)

# latitude is asin(z / R)
#   with rounding to prevent error in asin when |z/r| > 1
bead_lats = asin.( round(point[3]/icoso_circumradius, sigdigits=9) for point in points)

# longitude is atan2(y, x)
bead_lons = [atan(point[2],point[1]) for point in points]

degrees_lat = round.(rad2deg.(bead_lats), sigdigits=7)
degrees_lon = round.(rad2deg.(bead_lons))

earth12_labels = ["MA","BA","DA","LA","KA","SA","ME","BE","DE","LE","KE","SE"]

struct Bead
    location::LLA
    radius::Float64
    label::String
    description::String
end

expansion = 1 + 2π/5
contraction = 1 / (1 + 2π/5)

earth12 = []
contracted_radius = contraction * earth_radius
for i in 1:12    
    push!(earth12, Bead(LLA(degrees_lat[i],  degrees_lon[i]),
                        contracted_radius,
                        earth12_labels[i],
                        "point " * string(i) * " of 12 equidistant points"))
end
earth12


12-element Array{Any,1}:
 Bead(LLA(lat=-90.0°, lon=90.0°, alt=0.0), 2.826390255215226e6, "MA", "point 1 of 12 equidistant points")      
 Bead(LLA(lat=-26.56505°, lon=-72.0°, alt=0.0), 2.826390255215226e6, "BA", "point 2 of 12 equidistant points") 
 Bead(LLA(lat=-26.56505°, lon=144.0°, alt=0.0), 2.826390255215226e6, "DA", "point 3 of 12 equidistant points") 
 Bead(LLA(lat=-26.56505°, lon=72.0°, alt=0.0), 2.826390255215226e6, "LA", "point 4 of 12 equidistant points")  
 Bead(LLA(lat=-26.56505°, lon=-144.0°, alt=0.0), 2.826390255215226e6, "KA", "point 5 of 12 equidistant points")
 Bead(LLA(lat=-26.56505°, lon=0.0°, alt=0.0), 2.826390255215226e6, "SA", "point 6 of 12 equidistant points")   
 Bead(LLA(lat=90.0°, lon=-90.0°, alt=0.0), 2.826390255215226e6, "ME", "point 7 of 12 equidistant points")      
 Bead(LLA(lat=26.56505°, lon=108.0°, alt=0.0), 2.826390255215226e6, "BE", "point 8 of 12 equidistant points")  
 Bead(LLA(lat=26.56505°, lon=-36.0°, alt=0.0), 2.826390255215226e6, "DE", "poin

In [21]:
# pi/10 is 18 degrees, 3 * pi/10 is 54
# in geographic terms, the first element of each pair is North (Z+), the second is East (Y+)
directions = SVector.([ ( 0           ,  0            ), 
                        ( sin(pi/10)  , -cos(pi/10)   ), 
                        (-sin(3*pi/10), -cos(3*pi/10) ), 
                        (-sin(3*pi/10),  cos(3*pi/10) ),
                        ( sin(pi/10)  ,  cos(pi/10)   ), 
                        ( Φ           ,  0            )])
println("directions[]; SVector(s) of North,East tuples; in radians: \n",
        directions, "\n")

# NEED TO DO: get wgs84 meshed in, as results seem weird
# so doing:
#LinearMap(RotZ(direction[1])) ∘ LinearMap(RotY(direction[2])))
#LinearMap(RotY(direction[2])))
LinearMap(RotZ(directions[2][1]))


directions_degrees = [rad2deg.(direction) for direction in directions]
println("directions[] as above, but in degrees: \n",
        directions_degrees, "\n")

# these directions help form transformations for expanding and and contracting an area;
# nothing gets changed on X; Y is the East/second element; Z is the North/first element
transformations = [] 
for direction in directions
    push!(transformations, 
        LinearMap(RotZ(direction[1])) ∘ LinearMap(RotY(direction[2])))
end
# each point has its latitude and longitude calculated relative to the original point and the new radius 

labels = ["M","B","D","L", "K", "S"]

function contract(bead::Bead)
    beads = []
    radius = bead.radius * contraction
    #radius = bead.radius # * contraction
    ctr = bead.location
    for i in 1:6         
      push!(beads, 
        Bead(LLA(ctr.lat + rad2deg(directions[i][1]), 
                 ctr.lon + rad2deg(directions[i][2])), 
                 radius, 
                 bead.label * labels[i],
                 "" ))
    end
    return beads 
end

# my current longitude and latitude; altitude is set to 0.0 because it's unneeded
myspot = myspot_lla = LLA( 30.228711, 92.027100, 0.0 )
println("My Spot (LLA): \n", myspot,"\n")



# in Earth-Centred Earth-Fixed coordinates
myspot_ecef = ECEFfromLLA(wgs84)(myspot_lla)
println("Earth-Centred Earth-Fixed: \n",  myspot_ecef, "\n")

## composite = LinearMap(RotZ(directions[2][1])) ∘ 
##            LinearMap(RotY(directions[2][2]))
composite = transformations[2]
println("Composite Translation (North, East): \n",  composite, "\n")



### AAAARRRRRRGGGGHHHHH    ;)

# Latitude rotation occurs first
lat_rot = LinearMap(RotY(directions[2][2]))
println("Latitude rotation linear map: \n", lat_rot, "\n")

#ECEF(wgs84)
new_lat = lat_rot(myspot_ecef)
println("new_lat = lat_rot(myspot_ecef): \n", new_lat, "\n")
println(LLAfromECEF(wgs84)(new_lat))
newlat_lla = LLA(new_lat,wgs84)
#lat_rot(myspot_ecef)
println("new_lat_lla = nwlat #lat_rot(myspot_ecef): \n", newlat_lla, "\n")

# this doesn't work:
#my_trans_ecef = ECEF(composite(myspot_ecef))
## nor does this:
#my_trans_ecef = ECEF(wgs84)(composite(myspot_ecef))

#println("Composite translation of ecef: \n",  my_trans_ecef, "\n")

#my_trans_lla = LLAfromECEF(wgs84)(my_trans_ecef)
#println("New spot, as LLA: \n", my_trans_lla, "\n")
#trans(ECEF(myspot,wgs84))


#directions_degrees

#my_trans_ecef = trans(my_ecef)
#my_trans_lla = LLAfromECEF(wgs84)(my_trans_ecef)

#trans_lla_ecef = LLAfromECEF(wgs84)
#my_trans_lla = trans_lla_ecef(my_trans_ecef)
    
#LinearMap(RotZ(directions[2][1]))
#lat_rot

directions[]; SVector(s) of North,East tuples; in radians: 
SArray{Tuple{2},T,1,2} where T[[0, 0], [0.309017, -0.951057], [-0.809017, -0.587785], [-0.809017, 0.587785], [0.309017, 0.951057], [1.61803, 0.0]]

directions[] as above, but in degrees: 
SArray{Tuple{2},Float64,1,2}[[0.0, 0.0], [17.7054, -54.4915], [-46.3533, -33.6776], [-46.3533, 33.6776], [17.7054, 54.4915], [92.7065, 0.0]]

My Spot (LLA): 
LLA(lat=30.228711°, lon=92.0271°, alt=0.0)

Earth-Centred Earth-Fixed: 
[-1.95097e5, 5.51208e6, 3.19231e6]

Composite Translation (North, East): 
LinearMap([0.553312 -0.304122 -0.775471; 0.176641 0.952633 -0.247565; 0.81403 0.0 0.580823])

Latitude rotation linear map: 
LinearMap([0.580823 0.0 -0.81403; 0.0 1.0 0.0; 0.81403 0.0 0.580823])

new_lat = lat_rot(myspot_ecef): 
[-2.71195e6, 5.51208e6, 1.69535e6]



MethodError: MethodError: no method matching (::LLAfromECEF)(::SArray{Tuple{3},Float64,1,3})
Closest candidates are:
  LLAfromECEF(!Matched::ECEF) at /home/dhm/.julia/packages/Geodesy/qjth5/src/transformations.jl:58

In [174]:
center = bead.location
MA_point = LLA(-26.56505, -72.0, 0.0)
transformation = ENUfromLLA(center, wgs84)
#MA_location = LLAfromENU(transformation(MA_point),wgs84)
#LLAfromECEF(transformation(MA_point),wgs84)
enu = ECEF(center,wgs84)
#LLAfromENU(enu)
ma_vec = ENU(-72,26.56,0)
origin = ECEF(0,90,-90)
originenu = ENU(origin)

lfe = LLAfromECEF(wgs84)
lfe(origin)
efl = ECEFfromLLA(wgs84)
efl(lfe(origin))
lfn = LLAfromENU(center,wgs84)
#bead_1_location
#center,MA_point[1]
#transformation
#MA_location = LLAfromENU(transformation,wgs84)
#MA_location = LLAfromENU(transformation,wgs84)
lfn
ma_vec
originenu

#LLAfromENU(origin, datum) = LLAfromECEF(datum) ∘ ECEFfromENU(origin, datum)
#efenu = ECEFfromENU(origin,wgs84)
#LLAfromECEF(wgs84)
#trans = LLAfromENU(originenu,wgs84)

MA_point = LLA(-26.56505, -72.0, 0.0)
point_trans = ENUfromECEF(MA_point,wgs84)
originLLA = LLA(-90.0,90.0,0.0)
#originENU = ENUfromECEF(originLLA,wgs84)

#newpoint = ENU(MA_point,originLLA,wgs84)
#newpoint = ENU(originLLA,MA_point,wgs84)
#originENU(originLLA,wgs84)
trans = ENUfromLLA(MA_point,wgs84)
trans(MA_point)
v = LLA(20.,30.,0.)
vectrans = ENUfromECEF(v,wgs84)
#vectrans(originLLA,wgs84)
originLLA
o,m = originLLA , MA_point

newpoint = LLA(o.lat + m.lat, o.lon + m.lon, o.alt + m.alt)

LLA(lat=-116.56505°, lon=18.0°, alt=0.0)

In [204]:
myspot = LLA( 30.228711, 92.027100 )
ced404 = LLA( 30.228678, 92.027272 )
ber266 = LLA( 30.204461, 92.031397 )


using Printf
@printf "%.4f" distance(ced404, ber266)/1000

# for 3-D transformation; put into cartesian coordinates 
myspotECEF = ECEF( myspot, wgs84 )



2.7138

ECEF(-195096.58265667254, 5.512084465685038e6, 3.1923052497591996e6)